# Package import

In [14]:
import json
import requests
import ndjson
import pandas as pd
import os
from pandas_profiling import ProfileReport

# Functions

In [15]:
import ndjson
from typing import List, Dict

def load_ndjon(file_path: str) -> List[Dict]:
    with open(file_path) as f:
        return ndjson.load(f)

# Exploring the data

In [16]:
countries_df = pd.read_csv('../raw_data/countries.csv')
list(countries_df.country_code)

['GB', 'FR', 'NL']

In [17]:
data_path = os.path.join('..', 'raw_data', 'air_quality')

data = []
for filename in os.listdir(data_path):
    data.extend(load_ndjon(os.path.join(data_path, filename)))
    


In [18]:
df = pd.json_normalize(data, sep="_")

df.head()

,parameter,value,unit,location,city,country,attribution,sourceName,sourceType,mobile,date_utc,date_local,averagingPeriod_value,averagingPeriod_unit,coordinates_latitude,coordinates_longitude
0,pm25,-999.0,µg/m³,US Diplomatic Post: Kabul,Kabul,AF,"[{'name': 'EPA AirNow DOS', 'url': 'http://air...",StateAir_Kabul,government,False,2021-08-15T14:30:00.000Z,2021-08-15T19:00:00+04:30,1.0,hours,34.535812,69.190514
1,pm25,-999.0,µg/m³,US Diplomatic Post: Kabul,Kabul,AF,"[{'name': 'EPA AirNow DOS', 'url': 'http://air...",StateAir_Kabul,government,False,2021-08-15T15:30:00.000Z,2021-08-15T20:00:00+04:30,1.0,hours,34.535812,69.190514
2,pm25,-999.0,µg/m³,US Diplomatic Post: Kabul,Kabul,AF,"[{'name': 'EPA AirNow DOS', 'url': 'http://air...",StateAir_Kabul,government,False,2021-08-15T16:30:00.000Z,2021-08-15T21:00:00+04:30,1.0,hours,34.535812,69.190514
3,pm25,-999.0,µg/m³,US Diplomatic Post: Kabul,Kabul,AF,"[{'name': 'EPA AirNow DOS', 'url': 'http://air...",StateAir_Kabul,government,False,2021-08-15T17:30:00.000Z,2021-08-15T22:00:00+04:30,1.0,hours,34.535812,69.190514
4,pm25,-999.0,µg/m³,US Diplomatic Post: Kabul,Kabul,AF,"[{'name': 'EPA AirNow DOS', 'url': 'http://air...",StateAir_Kabul,government,False,2021-08-15T18:30:00.000Z,2021-08-15T23:00:00+04:30,1.0,hours,34.535812,69.190514


In [19]:
df.dtypes

parameter                 object
value                    float64
unit                      object
location                  object
city                      object
country                   object
attribution               object
sourceName                object
sourceType                object
mobile                      bool
date_utc                  object
date_local                object
averagingPeriod_value    float64
averagingPeriod_unit      object
coordinates_latitude     float64
coordinates_longitude    float64
dtype: object

In [20]:
df.parameter.unique()

array(['pm25', 'pm10', 'o3', 'no2', 'so2', 'co', 'bc'], dtype=object)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77273 entries, 0 to 77272
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   parameter              77273 non-null  object 
 1   value                  77273 non-null  float64
 2   unit                   77273 non-null  object 
 3   location               77273 non-null  object 
 4   city                   77273 non-null  object 
 5   country                77273 non-null  object 
 6   attribution            77273 non-null  object 
 7   sourceName             77273 non-null  object 
 8   sourceType             77273 non-null  object 
 9   mobile                 77273 non-null  bool   
 10  date_utc               77273 non-null  object 
 11  date_local             77273 non-null  object 
 12  averagingPeriod_value  77273 non-null  float64
 13  averagingPeriod_unit   77273 non-null  object 
 14  coordinates_latitude   77063 non-null  float64
 15  co

In [22]:
profile = ProfileReport(df, title="Air Quality data Report")
profile.to_file("air_quality_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]/home/alkis/.pyenv/versions/3.10.7/envs/openAQ/lib/python3.10/site-packages/multimethod/__init__.py:315: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  return func(*args, **kwargs)
Summarize dataset:  61%|██████    | 17/28 [00:03<00:01, 10.02it/s, Calculate auto correlation]             /home/alkis/.pyenv/versions/3.10.7/envs/openAQ/lib/python3.10/site-packages/multimethod/__init__.py:315: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  return func(*args, **kwargs)
Summarize dataset:  64%|██████▍   | 18/28 [00:05<00:00, 10.02it/s, Calculate spearman correlation]/home/alkis/.pyenv/versions/3.10.7/envs/openAQ/lib/python3.10/site-packages/multimethod/__init__.py:3

# Filtering the data

In [23]:
from enum import Enum, unique

@unique
class AirQualityIndex(Enum):
    PM25: str = 'pm25'
    PM10: str = 'pm10'
    O3: str = 'o3'
    NO2: str = 'no2'
    CO: str = 'co'
    
    @classmethod
    def to_list(cls):
        return [index.value  for index in cls.__members__.values()]
        
    

AirQualityIndex.to_list()
    

['pm25', 'pm10', 'o3', 'no2', 'co']

In [24]:



counties = list(countries_df.country_code)
paramaters = AirQualityIndex.to_list()

def filter_air_quality_data(df: pd.DataFrame, countries: List[str], paramaters: List[str]) -> pd.DataFrame:
    # columns_filter = ['parameter', 'value', 'city','date_utc', 'date_local', 'averagingPeriod_value', 'averagingPeriod_unit']
    filter = df['country'].isin(countries) & df['parameter'].isin(paramaters)
    return df[filter].copy()


def filter_by_parameter(df, parameter):
    return df[df['parameter'] == parameter].copy()



filtered_df = filter_air_quality_data(df, counties, paramaters)
filtered_df['date_utc'] = pd.to_datetime(filtered_df['date_utc'])
filtered_df['date_local'] = pd.to_datetime(filtered_df['date_local'])




# PM25

In [25]:

# filtered_df['date_utc'] = pd.to_datetime(filtered_df['date_utc'])
pm25_df = filter_by_parameter(filtered_df, AirQualityIndex.PM25.value)
pm25_df['day'] = pm25_df['date_utc'].dt.day


pm25_df

,parameter,value,unit,location,city,country,attribution,sourceName,sourceType,mobile,date_utc,date_local,averagingPeriod_value,averagingPeriod_unit,coordinates_latitude,coordinates_longitude,day
1599,pm25,1.0,µg/m³,Auchencorth Moss,Auchencorth,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 16:00:00+00:00,2021-10-06 17:00:00+01:00,24.0,hours,55.792160,-3.242900,6
1607,pm25,1.0,µg/m³,Edinburgh St Leonards,Edinburgh,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 16:00:00+00:00,2021-10-06 17:00:00+01:00,24.0,hours,55.945589,-3.182186,6
1611,pm25,3.0,µg/m³,Glasgow High Street,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 16:00:00+00:00,2021-10-06 17:00:00+01:00,24.0,hours,55.860936,-4.238214,6
1616,pm25,2.0,µg/m³,Glasgow Townhead,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 16:00:00+00:00,2021-10-06 17:00:00+01:00,24.0,hours,55.865782,-4.243631,6
1620,pm25,3.0,µg/m³,Grangemouth,Grangemouth,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 16:00:00+00:00,2021-10-06 17:00:00+01:00,24.0,hours,56.010319,-3.704399,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75965,pm25,-0.2,µg/m³,FR06003,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 09:00:00+00:00,2021-10-06 11:00:00+02:00,1.0,hours,50.378241,3.065860,6
75966,pm25,1.7,µg/m³,FR06003,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 10:00:00+00:00,2021-10-06 12:00:00+02:00,1.0,hours,50.378241,3.065860,6
75967,pm25,11.0,µg/m³,FR04156,Seine-Saint-Denis,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 09:00:00+00:00,2021-10-06 11:00:00+02:00,1.0,hours,48.902477,2.452500,6
75968,pm25,5.2,µg/m³,FR34014,Loiret,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 09:00:00+00:00,2021-10-06 11:00:00+02:00,1.0,hours,47.917375,1.966259,6


### Difference betweeen 24h and 1h

In [26]:
cities_24h_df = pm25_df[pm25_df['averagingPeriod_value'] == 24].city.unique()


In [27]:
cities_1h_df = pm25_df[pm25_df['averagingPeriod_value'] == 1].city.unique()


set(cities_1h_df).intersection(set(cities_24h_df))

{'Eastbourne', 'Reading'}

In [28]:
h1_df = pm25_df[pm25_df['averagingPeriod_value'] == 1]
h1_df

,parameter,value,unit,location,city,country,attribution,sourceName,sourceType,mobile,date_utc,date_local,averagingPeriod_value,averagingPeriod_unit,coordinates_latitude,coordinates_longitude,day
2181,pm25,2.4,µg/m³,Bexley - Slade Green Fidas,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 00:00:00+00:00,2021-10-06 00:00:00+00:00,1.0,hours,51.465983,0.184877,6
2182,pm25,2.2,µg/m³,Bexley - Slade Green Fidas,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 01:00:00+00:00,2021-10-06 01:00:00+00:00,1.0,hours,51.465983,0.184877,6
2183,pm25,2.4,µg/m³,Bexley - Slade Green Fidas,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 02:00:00+00:00,2021-10-06 02:00:00+00:00,1.0,hours,51.465983,0.184877,6
2184,pm25,2.5,µg/m³,Bexley - Slade Green Fidas,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 03:00:00+00:00,2021-10-06 03:00:00+00:00,1.0,hours,51.465983,0.184877,6
2185,pm25,2.6,µg/m³,Bexley - Slade Green Fidas,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 04:00:00+00:00,2021-10-06 04:00:00+00:00,1.0,hours,51.465983,0.184877,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75965,pm25,-0.2,µg/m³,FR06003,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 09:00:00+00:00,2021-10-06 11:00:00+02:00,1.0,hours,50.378241,3.065860,6
75966,pm25,1.7,µg/m³,FR06003,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 10:00:00+00:00,2021-10-06 12:00:00+02:00,1.0,hours,50.378241,3.065860,6
75967,pm25,11.0,µg/m³,FR04156,Seine-Saint-Denis,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 09:00:00+00:00,2021-10-06 11:00:00+02:00,1.0,hours,48.902477,2.452500,6
75968,pm25,5.2,µg/m³,FR34014,Loiret,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 09:00:00+00:00,2021-10-06 11:00:00+02:00,1.0,hours,47.917375,1.966259,6


## Testing a city

In [29]:
pm25_df.city.unique()

array(['Auchencorth', 'Edinburgh', 'Glasgow', 'Grangemouth',
       'Central Scotland', 'Chesterfield', 'Leicester', 'Nottingham',
       'Norwich', 'Sandy', 'London', 'Stanford-le-Hope', 'Inverness',
       'Middlesbrough', 'Newcastle', 'Stockton-on-Tees', 'Sunderland',
       'Wrexham', 'Blackpool', 'Manchester', 'Preston', 'Warrington',
       'Wigan', 'Liverpool', 'Belfast', 'Derry', 'Lough Navar', 'Chatham',
       'Stockbridge', 'Eastbourne', 'Oxford', 'Reading', 'Rochester',
       'Southampton', 'South East', 'Cardiff', 'Chepstow', 'Narberth',
       'Newport', 'Port Talbot', 'Swansea', 'Barnstaple', 'South West',
       'Plymouth', 'Saltash', 'West Midlands', 'Coventry',
       'Leamington Spa', 'Stoke-on-Trent', 'Hull', 'Leeds',
       'Yorkshire & Humberside', 'Sheffield', 'York', 'Bexley', 'Brent',
       'City of London', 'Crawley', 'Croydon', 'Greenwich', 'Lewisham',
       'Richmond', 'Newham', 'N/A', 'Doubs', 'Bouches-du-Rhône',
       'Loir-et-Cher', 'Eure-et-Loir', 'N

In [30]:
pm25_df[pm25_df['city'] == 'Amsterdam']

,parameter,value,unit,location,city,country,attribution,sourceName,sourceType,mobile,date_utc,date_local,averagingPeriod_value,averagingPeriod_unit,coordinates_latitude,coordinates_longitude,day
50219,pm25,1.4,µg/m³,Amsterdam-Einsteinweg,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN,6
50220,pm25,5.8,µg/m³,Amsterdam-Van Diemenstraat,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN,6
50221,pm25,2.1,µg/m³,Amsterdam-Vondelpark,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN,6
50222,pm25,0.5,µg/m³,Amsterdam-Westerpark,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN,6
50223,pm25,9.7,µg/m³,Amsterdam-Stadhouderskade,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN,6
50231,pm25,3.3,µg/m³,Amsterdam-Spaarnwoude,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN,6
50232,pm25,4.2,µg/m³,Amsterdam-Hoogtij,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,24.0,hours,NaN,NaN,6
63844,pm25,5.1,µg/m³,Amsterdam-Einsteinweg,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 17:00:00+00:00,2021-10-06 19:00:00+02:00,24.0,hours,NaN,NaN,6
63845,pm25,7.8,µg/m³,Amsterdam-Van Diemenstraat,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 17:00:00+00:00,2021-10-06 19:00:00+02:00,24.0,hours,NaN,NaN,6
63846,pm25,6.0,µg/m³,Amsterdam-Vondelpark,Amsterdam,NL,"[{'name': 'RIVM', 'url': 'http://www.lml.rivm....",Netherlands,government,False,2021-10-06 17:00:00+00:00,2021-10-06 19:00:00+02:00,24.0,hours,NaN,NaN,6


In [31]:
pm25_24h = pm25_df[(pm25_df['averagingPeriod_value'] == 24)]
pm25_24h[pm25_24h['city'] == 'Glasgow']
pm25_24h

,parameter,value,unit,location,city,country,attribution,sourceName,sourceType,mobile,date_utc,date_local,averagingPeriod_value,averagingPeriod_unit,coordinates_latitude,coordinates_longitude,day
1611,pm25,3.0,µg/m³,Glasgow High Street,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 16:00:00+00:00,2021-10-06 17:00:00+01:00,24.0,hours,55.860936,-4.238214,6
1616,pm25,2.0,µg/m³,Glasgow Townhead,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 16:00:00+00:00,2021-10-06 17:00:00+01:00,24.0,hours,55.865782,-4.243631,6
15087,pm25,3.0,µg/m³,Glasgow High Street,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 14:00:00+00:00,2021-10-06 15:00:00+01:00,24.0,hours,55.860936,-4.238214,6
15092,pm25,2.0,µg/m³,Glasgow Townhead,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 14:00:00+00:00,2021-10-06 15:00:00+01:00,24.0,hours,55.865782,-4.243631,6
26656,pm25,3.0,µg/m³,Glasgow High Street,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 17:00:00+00:00,2021-10-06 18:00:00+01:00,24.0,hours,55.860936,-4.238214,6
26661,pm25,2.0,µg/m³,Glasgow Townhead,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 17:00:00+00:00,2021-10-06 18:00:00+01:00,24.0,hours,55.865782,-4.243631,6
51530,pm25,3.0,µg/m³,Glasgow High Street,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 18:00:00+00:00,2021-10-06 19:00:00+01:00,24.0,hours,55.860936,-4.238214,6
51535,pm25,2.0,µg/m³,Glasgow Townhead,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 18:00:00+00:00,2021-10-06 19:00:00+01:00,24.0,hours,55.865782,-4.243631,6
65436,pm25,3.0,µg/m³,Glasgow High Street,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 13:00:00+00:00,2021-10-06 14:00:00+01:00,24.0,hours,55.860936,-4.238214,6
65441,pm25,2.0,µg/m³,Glasgow Townhead,Glasgow,GB,[{'name': 'Department for Environmental Food &...,DEFRA,government,False,2021-10-06 13:00:00+00:00,2021-10-06 14:00:00+01:00,24.0,hours,55.865782,-4.243631,6


In [32]:
bexley_df = pm25_df[pm25_df['city'] == 'Bexley']
# bexley_df = bexley_df.set_index('date_utc')
# bexley_df.groupby('date_utc')['value'].mean()
bexley_df

,parameter,value,unit,location,city,country,attribution,sourceName,sourceType,mobile,date_utc,date_local,averagingPeriod_value,averagingPeriod_unit,coordinates_latitude,coordinates_longitude,day
2181,pm25,2.4,µg/m³,Bexley - Slade Green Fidas,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 00:00:00+00:00,2021-10-06 00:00:00+00:00,1.0,hours,51.465983,0.184877,6
2182,pm25,2.2,µg/m³,Bexley - Slade Green Fidas,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 01:00:00+00:00,2021-10-06 01:00:00+00:00,1.0,hours,51.465983,0.184877,6
2183,pm25,2.4,µg/m³,Bexley - Slade Green Fidas,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 02:00:00+00:00,2021-10-06 02:00:00+00:00,1.0,hours,51.465983,0.184877,6
2184,pm25,2.5,µg/m³,Bexley - Slade Green Fidas,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 03:00:00+00:00,2021-10-06 03:00:00+00:00,1.0,hours,51.465983,0.184877,6
2185,pm25,2.6,µg/m³,Bexley - Slade Green Fidas,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 04:00:00+00:00,2021-10-06 04:00:00+00:00,1.0,hours,51.465983,0.184877,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67411,pm25,8.4,µg/m³,Greenwich - Falconwood FDMS,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 08:00:00+00:00,2021-10-06 08:00:00+00:00,1.0,hours,51.456300,0.085606,6
67412,pm25,8.1,µg/m³,Greenwich - Falconwood FDMS,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 09:00:00+00:00,2021-10-06 09:00:00+00:00,1.0,hours,51.456300,0.085606,6
67413,pm25,8.6,µg/m³,Greenwich - Falconwood FDMS,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 10:00:00+00:00,2021-10-06 10:00:00+00:00,1.0,hours,51.456300,0.085606,6
67414,pm25,7.1,µg/m³,Greenwich - Falconwood FDMS,Bexley,GB,[{'name': 'Environmental Research Group of Imp...,London Air Quality Network,government,False,2021-10-06 11:00:00+00:00,2021-10-06 11:00:00+00:00,1.0,hours,51.456300,0.085606,6


# PM10

In [33]:
pm10_df = filter_by_parameter(filtered_df, AirQualityIndex.PM10.value)
pm10_df[(pm10_df['city'] == 'Nord') & (pm10_df['location'] == "FR10034")]

,parameter,value,unit,location,city,country,attribution,sourceName,sourceType,mobile,date_utc,date_local,averagingPeriod_value,averagingPeriod_unit,coordinates_latitude,coordinates_longitude
8157,pm10,26.8,µg/m³,FR10034,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 13:00:00+00:00,2021-10-06 15:00:00+02:00,1.0,hours,51.024451,2.302149
8158,pm10,32.9,µg/m³,FR10034,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,1.0,hours,51.024451,2.302149
9771,pm10,20.8,µg/m³,FR10034,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 11:00:00+00:00,2021-10-06 13:00:00+02:00,1.0,hours,51.024451,2.302149
9772,pm10,26.4,µg/m³,FR10034,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 12:00:00+00:00,2021-10-06 14:00:00+02:00,1.0,hours,51.024451,2.302149
36493,pm10,26.8,µg/m³,FR10034,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 13:00:00+00:00,2021-10-06 15:00:00+02:00,1.0,hours,51.024451,2.302149
36494,pm10,32.9,µg/m³,FR10034,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 14:00:00+00:00,2021-10-06 16:00:00+02:00,1.0,hours,51.024451,2.302149
62493,pm10,35.8,µg/m³,FR10034,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 15:00:00+00:00,2021-10-06 17:00:00+02:00,1.0,hours,51.024451,2.302149
62494,pm10,33.9,µg/m³,FR10034,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 16:00:00+00:00,2021-10-06 18:00:00+02:00,1.0,hours,51.024451,2.302149
73351,pm10,16.5,µg/m³,FR10034,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 09:00:00+00:00,2021-10-06 11:00:00+02:00,1.0,hours,51.024451,2.302149
73352,pm10,21.6,µg/m³,FR10034,Nord,FR,"[{'name': 'EEA', 'url': 'http://www.eea.europa...",EEA France,government,False,2021-10-06 10:00:00+00:00,2021-10-06 12:00:00+02:00,1.0,hours,51.024451,2.302149


# Data models

In [34]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column
from sqlalchemy.sql.sqltypes import Integer, String

Base = declarative_base()

class DBSources(Base):
    __tablename__="source"
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(250), nullable=False)
    sourceId = Column(Integer)
    location = Column(Integer)
    